In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)

from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)


from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import os

2024-04-11 20:17:20.082076: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 20:17:20.086757: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 20:17:20.143801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 20:17:21.309718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = VGG16()

In [3]:
path = os.path.join("..",
                    "in",
                    "Tobacco3482")

In [27]:
data = []

In [1]:
labels = []

In [2]:
for folder in os.listdir(path):
    label_path = os.path.join(path, folder)
    labels.append(folder)
    for img in os.listdir(label_path):
        data.append(igm)

NameError: name 'os' is not defined

In [21]:
data = np.array(data)

In [22]:
type(data)

numpy.ndarray

In [26]:
data

array([['2023086360.jpg', 'ADVE'],
       ['2070500515_0516.jpg', 'ADVE'],
       ['tob19002.28_tob19002.30.jpg', 'ADVE'],
       ...,
       ['501725136_501725137.jpg', 'Letter'],
       ['515766856+-6858.jpg', 'Letter'],
       ['503856809_503856811.jpg', 'Letter']], dtype='<U27')